## 2.1 Prepare the Dataset

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

# 2.1 Load Dataset
(train_ds, test_ds), ds_info = tfds.load ('mnist', split =['train', 'test'], as_supervised = True, with_info = True)

print("ds_info: \n", ds_info)
# tfds.show_examples(train_ds, ds_info)

ds_info: 
 tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\prizl\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={

In [14]:
def new_target_fnc(ds, sequence_len):
        """
        Creates list of new targets by alternately adding and subtracting
        The first digit is added, the second subtracted, the third added, etc
        Parameters
        ----------
        ds : TensorFlowDataset
        original mnist dataset containg images and targets as a tuple.
        sequence_len : int
        indicates at which point the sum has to reset for the new sequence
        Returns
        -------
        l : list
        list containing the new targets
        """
        
        t = list()
        for i, v in enumerate(ds):
            if i % sequence_len == 0:
                t.append(v)
                
            elif i % 2 != 0:
                a = t[i-1] - v  
                t.append(a)

            elif i % 2 == 0:
                a = t[i-1] + v 
                t.append(a)
        return t

In [22]:
new_target_fnc([1,2,3,4], 3)

[1, -1, 2, 4]

In [41]:
# 2.2 Data Pipeline
def preprocess(dataset, batchsize, sequence_len):

    '''
    :param dataset: the dataset to be prepared for input into the network
    :param batchsize: the desired batchsize
    :return: 2 datasets, one each for each of the math problems defined (see below), created after the original database was preprocessed with the
    steps below
    '''

    # Step 1 - General Preprocessing

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

    # flatten the images into vectors
    # dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1,)), target))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

    # data = tf.data.Dataset.zip((dataset.shuffle(2000), dataset.shuffle(2000)))

    # print(dataset.shape)
    # The output of that lambda function should be a tuple of two tensors of shapes (num_images, height, width, 1) and (num_images, 1) or (num_images,)
    
    # Step 2 - Pairing Data Tuples & Respective Parameterized Targets

    # create a dataset that contains 2000 samples from the overall dataset paired with 2000 other samples
    # data = tf.data.Dataset.zip((dataset.shuffle(2000), dataset.shuffle(2000), dataset.shuffle(2000), dataset.shuffle(2000)))
    
    # create the dataset for the first math problem (a + b >= 5) - remembering to cast to int versus boolean!
    # first = data.map(lambda x1, x2, x3, x4: (x1[0], x2[0], x3[0], x4[0], x1[1]))
    # second = data.map(lambda x1, x2, x3, x4: (x1[0], x2[0], x3[0], x4[0], x1[1] - x2[1]))
    # third = data.map(lambda x1, x2, x3, x4: (x1[0], x2[0], x3[0], x4[0], x1[1] - x2[1] + x3[1]))
    # fourth = data.map(lambda x1, x2, x3, x4: (x1[0], x2[0], x3[0], x4[0], x1[1] - x2[1] + x3[1] - x4[1]))
    
    # list = [first, second, third, fourth]

    sequenc_len = 4
    
    dataset = dataset.map(lambda img, target: img, new_target_fnc(target, sequenc_len))

    # Step 3 - Batching & Prefetching
    # new = new_target_fnc(data, sequence_len)
    new = dataset.batch(batchsize)
    new = new.prefetch(tf.data.AUTOTUNE)

    # The shape of your tensors should be (batch, sequence-length, features). 
    return new

In [40]:

train_ds = preprocess(train_ds, 32, 4)
test_ds = preprocess(test_ds, 32, 4)

for img1, label in train_ds.take(1):
    print(img1.shape, label.shape)

# (bs, num_images, height, width, 1)

for img1, label in test_ds.take(1):
    print(img1.shape, label.shape)
    
# (bs, num_images, 1)

(32, 28, 28, 1) (32,)
(32, 28, 28, 1) (32,)


## 2.1.1 Finding the targets (instruction still in revision)

## 2.2 The CNN & LSTM Network

In [4]:
class BasicConv(tf.keras.Model):
    def __init__(self, batch, sequence_length, image):
        super(BasicConv, self).__init__()

        # input 32x32x3 with 3 as the color channels
        self.convlayer1 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu') # after this: 32x32x24
        # self.convlayer2 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu') # 32x32x24
        # self.convlayer3 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu') # 32x32x24

        # self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2) # 16x16x24

        # self.normlayer = tf.keras.layers.Normalization(axis=-1,mean=None,invert=False)

        # self.convlayer4 = tf.keras.layers.Conv2D(filters=72, kernel_size=3, padding='same', activation='relu') # 16x16x72
        # self.convlayer5 = tf.keras.layers.Conv2D(filters=72, kernel_size=3, padding='same', activation='relu') # 16x16x72
        # self.convlayer6 = tf.keras.layers.Conv2D(filters=72, kernel_size=3, padding='same', activation='relu') # 16x16x72

        self.global_pool = tf.keras.layers.GlobalAvgPool2D() # 1x1x72

        # self.out = tf.keras.layers.Dense(10, activation='softmax')

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.Adam()

        self.metrics_list = [
                    tf.keras.metrics.Mean(name="loss"),
                    tf.keras.metrics.BinaryAccuracy(name="acc"), # only for subtask 0, not for subtask 1
                    ]

    @tf.function
    def call(self, x):
        x = self.convlayer1(x) ## trying it out as simple as possible
        # x = self.convlayer2(x)
        # x = self.convlayer3(x)
        # x = self.pooling(x)
        # x = self.convlayer4(x)
        # x = self.convlayer5(x)
        # x = self.convlayer6(x)
        # x = self.global_pool(x)
        x = tf.keras.layers.TimeDistributed(self.global_pool())(x)

        # Once you have encoded all images as vectors, the shape of the tensor should be (batch, sequence-length, features), 
        # which can be fed to a non-convolutional standard LSTM.
        return x


    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    @tf.function
    def train_step(self, input):
        img, label = input

        with tf.GradientTape() as tape:
            prediction = self(img, training=True)
            loss = self.loss_function(label, prediction)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update loss metric
        self.metrics[0].update_state(loss)

        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction) # + tf.reduce_sum(self.losses)

        # return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, input):

        img, label = input

        prediction = self(img, training=False)
        loss = self.loss_function(label, prediction) # + tf.reduce_sum(self.losses)

        # update loss metric
        self.metrics[0].update_state(loss)

        # for accuracy metrics:
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction)

        # return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

## 2.3 LSTM AbstractRNNCell layer